In [1]:
from torchvision.datasets import CIFAR10
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader, Dataset
import torch
from torchvision import transforms
import torch.optim as optim
import torch.nn as nn



In [2]:
transform = transforms.Compose([
    transforms.ToTensor(),  # Converts images to PyTorch tensors and scales to [0, 1]
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))  # Normalizes to [-1, 1]
])

In [4]:
train_data = CIFAR10("data/train/", train=True, download=True, transform=transform)
test_data = CIFAR10("data/test/", train=False, download=True, transform=transform)

100%|██████████| 170M/170M [00:29<00:00, 5.84MB/s] 


Extracting data/train/cifar-10-python.tar.gz to data/train/


100%|██████████| 170M/170M [00:25<00:00, 6.62MB/s] 


Extracting data/test/cifar-10-python.tar.gz to data/test/


In [5]:
classes = test_data.classes

In [6]:
# Create DataLoaders
train_loader = DataLoader(train_data, shuffle=True, batch_size=8, num_workers=2)
test_loader = DataLoader(test_data, shuffle=False, batch_size=8, num_workers=2)

In [ ]:
import torch.nn as nn
import torch.nn.functional as F


class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 6, 5) # 6, 28, 28 color channel=3, out put channel(random)=6, kernel=5. input 32 pixel. [input(32)- kernel(5)]/strid(1) = 27 + 1 = 28 
        self.pool = nn.MaxPool2d(2, 2) # 6, 14, 14, after pooling, devide height(28)/2, weight(28)/2, nXn pool metrics 
        self.conv2 = nn.Conv2d(6, 16, 5) # 16, 10, 10 -> 16, 5,5
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = torch.flatten(x, 1) # flatten all dimensions except batch
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x


net = Net()

In [8]:
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=.9)
loss_func = nn.CrossEntropyLoss()

In [9]:
for epoch in range(1):

    tr_loss = 0.0

    for i, data in enumerate(train_loader):
        img, target = data

        optimizer.zero_grad()
        outputs = net(img)
        loss = loss_func(outputs, target)

        loss.backward()
        optimizer.step()

        tr_loss += loss.item()

    tr_loss /= len(train_loader)

    print(f"Epoch: {epoch}  Training Loss: {tr_loss:.4f}")


torch.Size([8, 16, 5, 5])
torch.Size([8, 400])
torch.Size([8, 16, 5, 5])
torch.Size([8, 400])
torch.Size([8, 16, 5, 5])
torch.Size([8, 400])
torch.Size([8, 16, 5, 5])
torch.Size([8, 400])
torch.Size([8, 16, 5, 5])
torch.Size([8, 400])
torch.Size([8, 16, 5, 5])
torch.Size([8, 400])
torch.Size([8, 16, 5, 5])
torch.Size([8, 400])
torch.Size([8, 16, 5, 5])
torch.Size([8, 400])
torch.Size([8, 16, 5, 5])
torch.Size([8, 400])
torch.Size([8, 16, 5, 5])
torch.Size([8, 400])
torch.Size([8, 16, 5, 5])
torch.Size([8, 400])
torch.Size([8, 16, 5, 5])
torch.Size([8, 400])
torch.Size([8, 16, 5, 5])
torch.Size([8, 400])
torch.Size([8, 16, 5, 5])
torch.Size([8, 400])
torch.Size([8, 16, 5, 5])
torch.Size([8, 400])
torch.Size([8, 16, 5, 5])
torch.Size([8, 400])
torch.Size([8, 16, 5, 5])
torch.Size([8, 400])
torch.Size([8, 16, 5, 5])
torch.Size([8, 400])
torch.Size([8, 16, 5, 5])
torch.Size([8, 400])
torch.Size([8, 16, 5, 5])
torch.Size([8, 400])
torch.Size([8, 16, 5, 5])
torch.Size([8, 400])
torch.Size([8

In [ ]:
torch.save(net.state_dict(), "trained_net.pth") # saving the weights and biases, the params actually
"""
in order to make it work, make a new instance of the
net architecture on which the model has been trained on
"""

In [88]:
trained_net = Net()
trained_net.load_state_dict(torch.load("trained_net.pth"))

/var/folders/2s/t87m92j14g9b1yw35n324r5w0000gn/T/ipykernel_60856/2188018614.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  trained_net.load_state_dict(torch.load("train

<All keys matched successfully>

In [99]:
correct = 0
total = 0

trained_net.eval()

with torch.no_grad():
    for data in test_loader:
        img, target = data
        outputs = trained_net(img)
        _, predicts = torch.max(outputs,1)
        total += target.size(0)
        correct += (predicts == target).sum().item()

accuracy = 100 * correct/total

print(f"Accuracy: {accuracy:.4f}")

Accuracy: 63.9400


In [108]:
# inference 

new_transform = transforms.Compose([
    transforms.Resize((32,32)),
    transforms.ToTensor(),
    transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))
])

def load_image(img_path):
    image = Image.open(img_path)
    image = new_transform(image)
    image = image.unsqueeze(0)
    return image

image_path = ["cat.jpg", "pln.jpg"]
images = [load_image(img) for img in image_path]

trained_net.eval()
with torch.no_grad():
    for img in images:
        out = trained_net(img)
        _, pred = torch.max(out,1)
        print(classes[pred.item()])



bird
airplane
